In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
pd.set_option('display.max_columns', None)
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from src.data_funcs import *
from src.model_funcs import *
from pyspark.sql.functions import explode

In [2]:
anime_df = pd.read_csv('data/anime.csv')
rating_df = pd.read_csv('data/rating.csv')
anime_meta = pd.read_csv('data/AnimeList_meta.csv')
users_meta = pd.read_csv('data/UserList_Meta.csv')
rating_df = rating_df[rating_df['rating']!=-1]

In [3]:
anime_full = full_anime_df(rating_df, anime_df, anime_meta)
anime_map = anime_full[['anime_id','name','title_english', 'type']]

## Model Based Matrix Factorization Recommenders System

## Manual train_test_split on users

In [4]:
filt = rating_df.groupby('user_id').count()['rating']
user_ids = filt[filt>50].reset_index()['user_id'].values
over_df = rating_df[rating_df['user_id'].isin(user_ids)]
remaining_df = rating_df[~rating_df['user_id'].isin(user_ids)]
over_df.groupby('user_id').count()['rating'].sort_values()
y=over_df['user_id']
X=over_df.drop(columns=['user_id'])
anime_train, anime_test, user_train, user_test = train_test_split(X, y, test_size = 0.20, random_state = 0, stratify=y)
train_over_split = pd.concat([anime_train, user_train],axis=1)
train = pd.concat([train_over_split, remaining_df], axis=0)
test = pd.concat([anime_test, user_test],axis=1)

/Users/sherryduong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [5]:
#old
# y=over_2_df['user_id']
# X=over_2_df.drop(columns=['user_id'])
# anime_train, anime_test, user_train, user_test = train_test_split(X, y, test_size = 0.25, random_state = 0, stratify=y)
# train_over2_split = pd.concat([anime_train, user_train],axis=1)
# train = pd.concat([train_over2_split, remaining_df], axis=0)
# test = pd.concat([anime_test, user_test],axis=1)

## Using Spark ALS

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [7]:
# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert a Pandas DF to a Spark DF
#spark_df = spark.createDataFrame(pandas_df) 

# Convert a Spark DF to a Pandas DF
#pandas_df = spark_df.toPandas()

In [8]:
all_spark = spark.createDataFrame(rating_df)
train_spark = spark.createDataFrame(train)
test_spark = spark.createDataFrame(test)
#Note to self: Add some cross validation

## Train Test Split with Pyspark

In [9]:
# train, test = all_spark.randomSplit([0.8, 0.2], seed=0)
train_data, val_data = train_spark.randomSplit([0.8, 0.2], seed=0)

In [10]:
train_data

DataFrame[anime_id: bigint, rating: bigint, user_id: bigint]

In [11]:
als_model = ALS(
    itemCol='anime_id',
    userCol='user_id',
    ratingCol='rating',
    nonnegative=True,    
    maxIter=20,
    regParam=0.1,
    rank=10) 
als_model.setColdStartStrategy("drop")

recommender = als_model.fit(train_data)

os.system("say 'Complete'") 

Py4JJavaError: An error occurred while calling o92.fit.
: org.apache.spark.SparkException: Job 3 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:933)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:931)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:931)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:2130)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2043)
	at org.apache.spark.SparkContext$$anonfun$stop$6.apply$mcV$sp(SparkContext.scala:1949)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1340)
	at org.apache.spark.SparkContext.stop(SparkContext.scala:1948)
	at org.apache.spark.SparkCon

In [ ]:
preds_train = recommender.transform(train_data)
preds_val = recommender.transform(val_data)

In [ ]:
predstrain_df = preds_train.toPandas()
predsval_df = preds_val.toPandas()

rmse_train = np.sqrt(mean_squared_error(predstrain_df['rating'],predstrain_df['prediction']))
rmse_train = np.sqrt(mean_squared_error(predsval_df['rating'],predsval_df['prediction']))

print(f'rmse is {pct_mean}% of mean')
os.system("say 'Complete'") 

## Inspecting the latent features

In [ ]:
anime_features = recommender.itemFactors
user_features = recommender.userFactors
#10
anime_features.take(1)

In [ ]:
predictions_df

## Tuning the model with crossvalidation & hyperparameter tuning

In [ ]:
# #Tuning the model with crossvalidation
# paramGrid = ParamGridBuilder() \
#     .addGrid(als_model.rank, [10]) \
#     .addGrid(als_model.maxIter, [20]) \
#     .addGrid(als_model.regParam, [0.1]) \
#     .build()

# evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
#                                 predictionCol='prediction')

# crossval = CrossValidator(estimator=als_model,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=evaluator,
#                           numFolds=3)

# cvModel = crossval.fit(train_data)

# preds_cv = cvModel.transform(test_data)
# preds_cvdf = preds_cv.toPandas()
# rmse_cv = np.sqrt(mean_squared_error(preds_cvdf['rating'],preds_cvdf['prediction']))
# pct_mean_cv = round(rmse_cv/(rating_df['rating'].mean())*100,0)
# print(f'rmse is {pct_mean_cv}% of mean')
# os.system("say 'Complete'") 

## Getting Recommendations For Certain Anime based on what other users that rated the anime highly also liked

In [ ]:
#movie recomendations
movieRecs = recommender.recommendForAllItems(10)
movieRecs.filter(movieRecs.anime_id==120).show()
#This returns the recommender user to watch the anime based on anime_id

In [ ]:
rec = movieRecs.filter(movieRecs.anime_id==120)
recs = rec.withColumn('recommendations', explode(rec.recommendations))
recs.select(col("recommendations").getItem(0)).show()

In [ ]:
rec_df = recs.toPandas()
rec_df

In [ ]:
rec_df['user_id_rec'] = rec_df['recommendations'].transform(lambda x: x[0])
rec_df['pred_rating'] = rec_df['recommendations'].transform(lambda x: x[1])
rec_df

In [ ]:
similar_users = rec_df['user_id_rec'].values() 

In [ ]:
userRecs = recommender.recommendForAllUsers(10)
user_rec = userRecs.filter(userRecs.user_id==57805)
user_rec.show()

In [ ]:
recs = user_rec.withColumn('recommendations', explode(user_rec.recommendations))
recs.show()

In [ ]:
rec_df = recs.toPandas()
rec_df['anime_id_rec'] = rec_df['recommendations'].transform(lambda x: x[0])
rec_df['pred_rating'] = rec_df['recommendations'].transform(lambda x: x[1])
rec_df

In [ ]:
anime_map[anime_map['anime_id'].isin(rec_df['anime_id_rec'].values)]

In [ ]:
#Generate Recommendations
def get_recs_for_user(recs):
#recs should be for a specific user.
recs = recs.select(“recommendations.movieID”, “recommendations.rating”)
movies = recs.select(“movieID”).toPandas().iloc[0,0]
ratings = recs.select(“rating”).toPandas().iloc[0,0]
ratings_matrix = pd.DataFrame(movies, columns = [“movieId”]_
ratings_matrix[“ratings”] = ratings
ratings_matrix_ps = sql.Context.createDataFrame(ratings_matrix)
return ratings_matrix_ps

## Surprise Package

In [ ]:
from surprise import SVDpp, NormalPredictor, Dataset, Reader, accuracy
from surprise import KNNBaseline, NMF, SVD
from surprise.model_selection import KFold, cross_validate, GridSearchCV, train_test_split

In [ ]:
reader = Reader(rating_scale=(1, 10))
all_data = Dataset.load_from_df(rating_df, reader)
train_surprise = Dataset.load_from_df(train, reader)
test_surprise = Dataset.load_from_df(test, reader)

In [ ]:
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(train_surprise):

    # train and test algorithm.
    algo.fit(trainset)
    preds_surprise = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(preds_surprise, verbose=True)

In [ ]:
os.system("say 'Model Complete'") 

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49618)
Traceback (most recent call last):
  File "/Users/sherryduong/anaconda3/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/sherryduong/anaconda3/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/sherryduong/anaconda3/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/sherryduong/anaconda3/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/Users/sherryduong/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/Users/sherryduong/anaconda3/lib/python3.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/Users/sherryd